In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.

In [7]:
# An ADC (Analog to Digital Converter) is needed to read the value from each green light pulse sensor
# Not every pin on the ESP32 has this capability.

# Here we test which Pins can do ADC
import machine

for i in range(50):
    try:
        machine.ADC(machine.Pin(i))
        print("ADC works on Pin", i)
    except ValueError:
        pass
    

ADC works on Pin 32
ADC works on Pin 33
ADC works on Pin 34
ADC works on Pin 35
ADC works on Pin 36
ADC works on Pin 37
ADC works on Pin 38
ADC works on Pin 39


In [ ]:
# With the USB socket pointing away from you, the only pins that will 
# be good for this are on the right hand side, and are D33, D32, D35, D34

In [2]:
# we need to set the working range to between 0 and 5V using the atten() function
a34 = machine.ADC(machine.Pin(34))
a34.atten(machine.ADC.ATTN_11DB)

In [3]:
# We can make a short set of readings, like so:
import time

for i in range(10):
    print(a34.read())
    time.sleep(0.1)

1857
1847
1861
1858
1861
1845
1857
1862
1857
1850


In [20]:
# We can efficiently print out the readings (without overloading the browser 
# by printing it in an unlimited stream), like so:

import time

for i in range(60):
    for j in range(100):
        print(a34.read(), end=" ")  # puts a blank instead of a newline
        time.sleep(0.01)  # make a reading every 1/100th of a second
    print()

# You can stop the printing by selecting Kernel -> Interrupt in the menu, 
# or hitting escape and then "i" twice.

1836 1836 1836 1837 1835 1840 1840 1835 1852 1840 1838 1838 1835 1835 1835 1834 1827 1821 1819 1811 1823 1819 1829 1837 1799 1838 1835 1840 1845 1842 1838 1840 1835 1833 1824 1829 1819 1825 1823 1815 1814 1820 1822 1819 1828 1827 1831 1837 1842 1823 1857 1858 1858 1861 1858 1855 1872 1871 1873 1872 1870 1874 1871 1872 1874 1877 1877 1866 1869 1870 1856 1866 1856 1844 1840 1845 1840 1837 1837 1840 1829 1831 1838 1839 1841 1835 1840 1843 1829 1853 1837 1843 1850 1856 1856 1856 1856 1851 1856 1845 
1840 1853 1847 1849 1842 1849 1841 1842 1845 1840 1840 1843 1840 1841 1847 1834 1840 1839 1840 1838 1842 1843 1854 1850 1858 1858 1859 1852 1853 1850 1842 1843 1831 1841 1835 1793 1834 1833 1834 1835 1840 1845 1849 1841 1842 1850 1843 1845 1851 1842 1846 1845 1846 1845 1847 1775 1873 1825 1846 1847 1851 1853 1858 1857 1862 1851 1863 1856 1859 1862 1859 1856 1861 1862 1839 1834 1855 1855 1856 1855 1847 1855 1856 1854 1851 1849 1853 1850 1846 1841 1846 1840 1837 1834 1839 1838 1833 1838 1832 1837

1850 1840 1846 1840 1855 1845 1840 1854 1844 1831 1843 1833 1849 1842 1853 1840 1847 1832 1851 1836 1850 1844 1855 1837 1846 1839 1856 1840 1845 1834 1840 1834 1856 1830 1851 1834 1847 1837 1845 1834 1847 1843 1847 1835 1850 1847 1847 1831 1842 1831 1840 1837 1847 1839 1840 1833 1840 1851 1846 1842 1847 1845 1853 1851 1842 1840 1840 1842 1841 1845 1837 1845 1834 1833 1830 1849 1829 1847 1841 1840 1831 1855 1842 1850 1835 1845 1833 1847 1835 1852 1835 1846 1838 1852 1838 1855 1835 1847 1839 1846 
1840 1851 1840 1846 1838 1840 1841 1849 1840 1847 1840 1837 1837 1840 1840 1841 1837 1849 1840 1844 1845 1840 1838 1845 1836 1840 1842 1843 1846 1834 1855 1842 1843 1838 1839 1840 1854 1834 1851 1831 1854 1840 1855 1834 1853 1835 1853 1843 1847 1840 1851 1838 1841 1835 1854 1838 1847 1835 1843 1841 1843 1823 1853 1835 1853 1835 1858 1839 1846 1839 1846 1840 1852 1841 1852 1840 1843 1835 1846 1840 1843 1841 1841 1842 1843 1850 1842 1843 1862 1841 1815 1844 1840 1838 1839 1852 1834 1849 1835 1860

Traceback (most recent call last):
  File "<stdin>", line 9, in <module>
KeyboardInterrupt: 


In [9]:
print(time.ticks_ms())


2908109


In [ ]:
%capture --quiet testpulsesingle.txt

# Make 15 seconds of samples at 100 samples per second
import time

for i in range(15):
    for j in range(100):
        print(a34.read(), end=" ")  # puts a blank instead of a newline
        time.sleep(0.01)
    print()
    

In [ ]:
# To plot what these readings are, you need to open up the 
# notebook "plot_adc_measurements" which runs in Python on your PC
# (instead of the Python in the ESP32) and allows you to do scientific plots


In [12]:
import time

def measure34(numberofseconds, samplespersecond):
    ticks0 = time.ticks_ms()
    for i in range(numberofseconds):
        for j in range(samplespersecond):
            print(a34.read(), end=" ")  # puts a blank instead of a newline
            time.sleep(1/samplespersecond)
        print()
    return (time.ticks_ms() - ticks0)/1000
    

In [18]:
%capture --quiet testpulsesingle100.txt
elapsedtime = measure34(10, 100)

10 lines captured

In [19]:
print(elapsedtime, "should be close to", numberofseconds)

10.208 should be close to 10


In [20]:
%capture --quiet testpulsesingle500.txt
elapsedtime = measure34(10, 500)

10 lines captured

In [21]:
print(elapsedtime, "should be close to", numberofseconds)

11.034 should be close to 10


In [ ]:
# There seems to be high frequency noise in the sensor
# So this lets us make 10 readings at a time and then 
# print them out at a frequency of 50 samples per second
# which gives a pretty smooth result

In [28]:
import time

def measure34sub(numberofseconds, samplespersecond, subsamplestoaverage):
    ticks0 = time.ticks_ms()
    microsecondstep = int(1/samplespersecond/subsamplestoaverage*1000000)
    for i in range(numberofseconds):
        for j in range(samplespersecond):
            samplesum = 0
            for k in range(subsamplestoaverage):
                samplesum += a34.read()
                time.sleep_us(microsecondstep)
            print(int(samplesum/subsamplestoaverage+0.5), end=" ")
        print()
    return (time.ticks_ms() - ticks0)/1000
    

In [29]:
%capture --quiet testpulsesingle50avg.txt
elapsedtime = measure34sub(10, 50, 10)


10 lines captured

In [30]:
print(elapsedtime, "should be close to", numberofseconds)

10.262 should be close to 10
